In [6]:
import pandas as pd
from sqlalchemy import create_engine

In [7]:
# EXTRACT DATA

df1=pd.read_csv("steam_games.csv")

df2=pd.read_csv("steam-200k.csv", header=None)

df3=pd.read_csv("games-features.csv")

In [34]:
# TRANSFORM DATASETS

newdf2 = df2[[0, 1, 2, 3]].copy()
dfdf=newdf2.drop(newdf2.index[0])
aa=dfdf.columns=['userid','gamename', 'subscriptiontype', 'hours']
aa=dfdf[dfdf.subscriptiontype != 'purchase']
data2=aa.drop('userid', axis=1)
data2
bb=aa.groupby(['gamename'])
cc=bb['hours'].count()
dd=cc.sort_values(ascending=False)
ee=pd.DataFrame(dd)
ff=ee.reset_index()
ff=ff.reset_index()
gg=ff.rename(columns={"index": "gameid", "gamename": "gamename", "hours": "gamehours"})


gg2=df3[['ResponseName','Metacritic','RecommendationCount','IsFree','PCReqsHaveMin','PriceInitial']]
gg2.columns=['gamename','metacritic', 'reccount', 'IsFree', 'PCRECs', 'Price']
gg3=pd.merge(gg,gg2, how='inner',on='gamename')
#please remove duplicates from gg3


zz=df1[['name', 'developer', 'publisher', 'popular_tags', 'minimum_requirements', 'original_price']]
zz.columns=['gamename', 'developers', 'publisher', 'popular_tags', 'minreqs', 'price']
zz2=pd.merge(zz,gg3, how='inner',on='gamename')

print(len(gg))
print(len(gg2))
print(len(gg3))
print(len(zz2))

3600
13357
1942
1460


In [37]:
# LOADING DATA INTO SQL 

# CONNECTION TO MYSQL

rds_connection_string = "postgres:postgres@localhost:5432/ETL PROJECT"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [39]:
engine.table_names()

['publisherdata2', 'mostplayedgames2', 'steamrecs2']

In [ ]:
# gg2.to_sql(name='steamrecs2', con=engine, if_exists='append', index=False)
# gg.to_sql(name='mostplayedgames2', con=engine, if_exists='append', index=False)
# zz.to_sql(name='publisherdata2', con=engine, if_exists='append', index=False)

In [41]:
# TEST TO SEE IF CONNECTION IS WORKING

pd.read_sql_query('select * from steamrecs2', con=engine).head()

,gamename,metacritic,reccount,isfree,pcrecs,price
0,Counter-Strike,88,68991,false,true,9.99
1,Team Fortress Classic,0,2439,false,true,4.99
2,Day of Defeat,79,2319,false,true,4.99
3,Deathmatch Classic,0,888,false,true,4.99
4,Half-Life: Opposing Force,0,2934,false,true,4.99


In [42]:
# Continue testing 


pd.read_sql_query("""
SELECT * from mostplayedgames2 as a
LEFT JOIN steamrecs2 as b
on a.gamename=b.gamename
LEFT JOIN publisherdata2 as c
on a.gamename=c.gamename


""", con=engine).head()

,gameid,gamename,gamehours,gamename,metacritic,reccount,isfree,pcrecs,price,gamename,developers,publisher,popular_tags,minreqs,price
0,26,DayZ,246,DayZ,0,153350,false,true,34.99,DayZ,Bohemia Interactive,"Bohemia Interactive,Bohemia Interactive","Survival,Zombies,Open World,Multiplayer,PvP,Ma...","Minimum:,OS:,Windows 7/8.1 64-bit,Processor:,I...",$44.99
1,621,EVE Online,21,EVE Online,88,4269,false,true,19.99,EVE Online,CCP,"CCP,CCP","Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","Minimum:,OS:,Windows 7,Processor:,Intel Dual C...",Free
2,29,Grand Theft Auto IV,226,Grand Theft Auto IV,90,36012,false,true,19.99,Grand Theft Auto IV,"Rockstar North,Rockstar Toronto","Rockstar Games,Rockstar Games","Open World,Action,Bowling,Multiplayer,Third Pe...",0,$19.99
3,13,Portal,417,Portal,90,27546,false,true,9.99,Portal,Valve,"Valve,Valve","Puzzle,First-Person,Singleplayer,Sci-fi,Comedy...",0,$9.99
4,13,Portal,417,Portal,90,27546,false,true,9.99,Portal,Valve,"Valve,Valve","Puzzle,First-Person,Singleplayer,Sci-fi,Comedy...",0,$9.99
